In [1]:
pip install nba_api

     ------------------------------------ 125.6/125.6 kB 670.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 62.8/62.8 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 160.4/160.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 140.4/140.4 kB 1.7 MB/s eta 0:00:00
Using legacy 'setup.py install' for nba_api, since package 'wheel' is not installed.
  Running setup.py install for nba_api: started
  Running setup.py install for nba_api: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [3]:
gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable='01/31/2020', league_id_nullable='00')
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
2,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
3,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
4,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.737,11,31,42,22,8,6,15,17,-10.0


## Clean and Explore data

In [4]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [5]:
games = games[['TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'PLUS_MINUS']]

In [6]:
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS
0,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,-13.0
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0
...,...,...,...,...,...,...
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0
6150,Los Angeles Lakers,0021900726,2020-01-31,LAL vs. POR,L,-8.0


final DataFrame: two columns: 1.result of game: target 2.score statistics comparing two teams: feature

In [7]:
import pandas as pd

In [8]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6152 entries, 0 to 6151
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM_NAME   6152 non-null   object 
 1   GAME_ID     6152 non-null   object 
 2   GAME_DATE   6152 non-null   object 
 3   MATCHUP     6152 non-null   object 
 4   WL          6152 non-null   object 
 5   PLUS_MINUS  6152 non-null   float64
dtypes: float64(1), object(5)
memory usage: 288.5+ KB


In [9]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])

In [10]:
games['GAME_DATE']

0      2022-06-16
1      2022-06-16
2      2022-06-13
3      2022-06-13
4      2022-06-10
          ...    
6147   2020-01-31
6148   2020-01-31
6149   2020-01-31
6150   2020-01-31
6151   2020-01-31
Name: GAME_DATE, Length: 6152, dtype: datetime64[ns]

In [12]:
games = games.sort_values('GAME_DATE')
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS
6151,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0
6146,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0
...,...,...,...,...,...,...
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0


In [14]:
games['avg_30_plus_minus'] = games.groupby('TEAM_NAME')['PLUS_MINUS'].transform(lambda x: x.rolling(30, closed='left').mean())


In [15]:
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS,avg_30_plus_minus
6151,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0,NaN
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0,NaN
6146,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0,NaN
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0,NaN
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0,NaN
...,...,...,...,...,...,...,...
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0,8.366667
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0,7.100000
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0,3.900000
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0,3.766667
